# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
import re
import pickle
import nltk
import warnings
nltk.download('punkt')
nltk.download('stopwords')
warnings.simplefilter('ignore')
from sklearn.svm import SVC
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///MessagesETL.db')
df = pd.read_sql("SELECT * FROM MessagesETL", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    # Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize words
    tokens = word_tokenize(text)    
    
    # remove stop words
    tokens = [w for w in tokens if w not in stopwords.words("english")] 
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]    
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
new_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

new_pipeline_fitted = new_pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_prediction_test = new_pipeline_fitted.predict(X_test)

In [7]:
# Print classification report on test data
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_prediction_test[:, i]))

related
             precision    recall  f1-score   support

          0       0.61      0.46      0.52      1509
          1       0.85      0.91      0.88      5045

avg / total       0.79      0.81      0.80      6554

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5395
          1       0.80      0.43      0.56      1159

avg / total       0.87      0.88      0.87      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6522
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.75      0.84      0.79      3810
          1       0.74      0.61      0.66      2744

avg / total       0.74      0.74      0.74      6554

medical_help
             precision    recall  f1-score   support

          0       0.92      0.99      0

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters =  {'tfidf__use_idf': (True, False), 
              'clf__estimator__n_estimators': [10, 20], 
              'clf__estimator__min_samples_split': [2, 4]} 

cv = GridSearchCV(new_pipeline, param_grid=parameters)
cv

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 20], 'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [10]:
Y_predictionCV_test = cv.predict(X_test)

In [11]:
# Print classification report on test data
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_predictionCV_test[:, i]))

related
             precision    recall  f1-score   support

          0       0.69      0.43      0.53      1509
          1       0.85      0.94      0.89      5045

avg / total       0.81      0.82      0.81      6554

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      5395
          1       0.83      0.43      0.57      1159

avg / total       0.88      0.88      0.87      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6522
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.78      0.83      0.80      3810
          1       0.74      0.67      0.71      2744

avg / total       0.76      0.77      0.76      6554

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
from sklearn.decomposition import TruncatedSVD
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

new_pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('best', TruncatedSVD()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [13]:
new_pipeline2_fitted = new_pipeline2.fit(X_train, Y_train)

In [14]:
Y_prediction2_test = new_pipeline2_fitted.predict(X_test)

In [15]:
# Print classification report on test data
for i, col in enumerate(Y_test):
    print(col)
    print(classification_report(Y_test[col], Y_prediction2_test[:, i]))

related
             precision    recall  f1-score   support

          0       0.67      0.00      0.01      1509
          1       0.77      1.00      0.87      5045

avg / total       0.75      0.77      0.67      6554

request
             precision    recall  f1-score   support

          0       0.82      1.00      0.90      5395
          1       0.50      0.00      0.00      1159

avg / total       0.77      0.82      0.74      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6522
          1       0.00      0.00      0.00        32

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.58      1.00      0.74      3810
          1       0.33      0.00      0.00      2744

avg / total       0.48      0.58      0.43      6554

medical_help
             precision    recall  f1-score   support

          0       0.92      1.00      0

### 9. Export your model as a pickle file

In [16]:
with open('model.pkl', 'wb') as f:
    pickle.dump(new_pipeline2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.